In [912]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns

from sklearn import (linear_model, metrics, neural_network, pipeline, preprocessing, model_selection)

In [913]:
"""
Load data.
Change path variable to your location of CSV files.
"""

path = "./data/"

df_zones = pd.read_csv(path + "zona_censal_comuna.csv")
df_demos = pd.read_csv(path + "age_demo_comunas.csv")
df_inf = pd.read_csv(path + "dp1_contagiados_por_comuna.csv")
df_mob1 = pd.read_csv(path + "od_zones_weeks_20200301.enc.csv")
df_mob2 = pd.read_csv(path + "od_zones_weeks_20200401.enc.csv")
df_mob = pd.concat([df_mob1, df_mob2])

### Census zones & Communes

In [914]:
"""
Census zone information (only for Santiago).

Relevant columns:
    GEOCODIGO = census zone id
    COMUNA    = commune id
"""
df_zones

,OBJECTID,REGION,PROVINCIA,NOM_COMUNA,COMUNA,GEOCODIGO
0,1,13,134,PAINE,13404,13404011003
1,2,13,134,PAINE,13404,13404011002
2,3,13,134,PAINE,13404,13404061004
3,4,13,134,PAINE,13404,13404061001
4,5,13,134,PAINE,13404,13404061002
...,...,...,...,...,...,...
1860,1861,13,136,EL MONTE,13602,13602031001
1861,1862,13,136,TALAGANTE,13601,13601021001
1862,1863,13,136,TALAGANTE,13601,13601021002
1863,1864,13,135,ALHUÉ,13502,13502011001


In [915]:
"""
Create dictionaries for (standardized) ID lookups of zones and communes.
"""
# Enumerate communes with standardized IDs (from 0 to 50)
dict_commune_sid = {commune:sid for sid, commune in enumerate(df_zones.COMUNA.unique())}

# Dictionary mapping census zone id to its commune standardized id
dict_zone_commune_sid = {zone:dict_commune_sid[commune] for zone, commune in zip(df_zones.GEOCODIGO, df_zones.COMUNA)}    

### Mobility data

In [916]:
"""
Logs of averaged amount of movement of individuals across census zones.

Relevant columns:
    mean_population_zone = weekly mean of daily number of individuals
                                that spend majority of their day in census zone [code_zone]
                                that live in census zone [home_zone]
    code_zone            = visiting census zone
    home_zone            = home census zone
    week                 = week of year
    time_block           = T1 - 10:00-13:00, T2 - 14:00-17:00
"""
df_mob

,mean_population_zone,code_zone,week,time_block,home_zone
0,154.800000,13124061022,9,T1,13124061005
1,14.200000,13101081001,9,T1,13126031003
2,12.800000,13123021005,9,T1,13120041002
3,26.800000,13114021002,9,T1,13114061001
4,44.200000,13114131005,9,T1,13114031002
...,...,...,...,...,...
4642958,1.666667,13130031001,17,T2,13112051010
4642959,1.666667,13119011005,17,T2,13106051001
4642960,1.666667,13301011001,17,T1,13104051001
4642961,1.666667,13119201001,17,T2,13124061005


In [917]:
"""
Create dataframe of edge information: (home zone, visiting zone, week, mean population)
Aggregate information into communes.
"""
t = df_mob.week.min()    # earliest week available
get_zone_com_sid = lambda i: dict_zone_commune_sid[i]

df_edge_list = df_mob[["home_zone","code_zone","week","mean_population_zone"]]
df_edge_list = df_edge_list[df_edge_list["home_zone"] != df_edge_list["code_zone"]]    # remove intra-zone info
df_edge_list.columns = ['home_com', 'travel_com', 'week', 'mean_pop']
df_edge_list.loc[:,"home_com"] = df_edge_list.loc[:,"home_com"].apply(get_zone_com_sid)
df_edge_list.loc[:,"travel_com"] = df_edge_list.loc[:,"travel_com"].apply(get_zone_com_sid)
df_edge_list.loc[:,"week"] = df_edge_list.loc[:,"week"] - t    # zero index weeks

# Aggregate data by summing mean population of same edges in same week. Sum data from both time blocks.
df_edge_list = df_edge_list.groupby(['home_com','travel_com','week']).agg({'mean_pop':'sum'}).reset_index()

In [918]:
"""
Create mobility matrix: A[i,j,t] 
"""
T = df_mob.week.max() - df_mob.week.min() + 1
N = len(dict_commune_sid)
A = np.zeros((N, N, T))

edge_list = zip(df_edge_list.home_com, df_edge_list.travel_com, df_edge_list.week, df_edge_list.mean_pop)
for edge in edge_list:
    i, j, t, a_ijt = edge
    A[i,j,t] += a_ijt

In [919]:
# Mobility matrix over 51 communes in Santiago over 9 time periods (weeks 9-17).
print(A.shape)

# Matrix is not sparse
print(np.count_nonzero(A), N*N*T)

(51, 51, 9)
21931 23409


### Infections data

In [920]:
"""
Logs of cumulative number of infections in each commune in Chile over differently spaced out days in weeks 14-19.

Relevant columns:
    Codigo comuna          = commune ID
    Date (e.g. 2020-03-30) = number of confirmed cases of COVID19 on that day.
"""
df_inf

,Unnamed: 0,Region,Codigo region,Comuna,Codigo comuna,Poblacion,2020-03-30,2020-04-01,2020-04-03,2020-04-06,...,2020-04-13,2020-04-15,2020-04-17,2020-04-20,2020-04-24,2020-04-27,2020-05-01,2020-05-04,2020-05-08,Tasa
0,0,Arica y Parinacota,15,Arica,15101,247552.0,6.0,6.0,12.0,41.0,...,115.0,124.0,134.0,166.0,224.0,270.0,297.0,310.0,328.0,132.5
1,1,Arica y Parinacota,15,Camarones,15102,1233.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Arica y Parinacota,15,General Lagos,15202,810.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Arica y Parinacota,15,Putre,15201,2515.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Tarapacá,1,Alto Hospicio,1107,129999.0,0.0,0.0,0.0,5.0,...,14.0,15.0,16.0,27.0,39.0,55.0,77.0,128.0,161.0,123.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,341,Magallanes,12,Punta Arenas,12101,141984.0,29.0,87.0,143.0,203.0,...,387.0,416.0,470.0,516.0,581.0,623.0,685.0,744.0,825.0,581.1
342,342,Magallanes,12,Rio Verde,12103,211.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
343,343,Magallanes,12,San Gregorio,12104,681.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,146.8
344,344,Magallanes,12,Timaukel,12303,282.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [921]:
"""
Aggregate infections data into weeks.
Only keep data for communes in Santiago.
"""
get_com_sid = lambda i: dict_commune_sid[i] if i in dict_commune_sid else pd.NaT
df_y = pd.DataFrame()
df_y["com"] = df_inf["Codigo comuna"]
df_y.loc[:,"com"] = df_y.loc[:,"com"].apply(get_com_sid)
df_y["week14"] = df_inf["2020-03-30"] + df_inf["2020-04-01"] + df_inf["2020-04-03"]
df_y["week15"] = df_inf["2020-04-06"] + df_inf["2020-04-08"] + df_inf["2020-04-10"]
df_y["week16"] = df_inf["2020-04-13"] + df_inf["2020-04-15"] + df_inf["2020-04-17"]
df_y["week17"] = df_inf["2020-04-20"] + df_inf["2020-04-24"]
df_y["week18"] = df_inf["2020-04-27"] + df_inf["2020-05-01"]
df_y["week19"] = df_inf["2020-05-04"] + df_inf["2020-05-08"]
df_y = df_y.dropna().sort_values("com")

# Get time-evolving cumulative infections vector
y = df_y.drop("com",1).to_numpy()

# Transform data to ensure positivity in regression
# y = np.log(y + 1)

In [922]:
# Cumulative infections over 51 communes in Santiago over 6 time periods (weeks 14-19)
y.shape

(51, 6)

### Demographics

In [923]:
"""
Demographics information for all communes in Chile.
"""
df_demos.head()

,COMUNA,NOM_COMUNA,Densidad_,SUPERFICIE__KM2_,T_POB,T_HOM,T_MUJ,T_VIV,por_muj,por_hom,...,55 a 59,60 a 64,65 a 69,70 a 74,75 a 79,80 a 84,85 a 89,90 a 94,95 a 99,100 o más
0,13101,SANTIAGO,17483.935547,23.135237,404495,206678,197817,193628,0.489047,0.510953,...,16473,12820,9465,7449,5328,3738,2602,1027,302,108
1,13102,CERRILLOS,4817.263672,16.779650,80832,39631,41201,24547,0.509712,0.490288,...,4510,3602,2963,2550,1862,1154,738,267,57,18
2,13103,CERRO NAVIA,11950.771484,11.097359,132622,65438,67184,38020,0.506583,0.493417,...,7466,6013,5340,4550,3160,1842,1028,340,103,34
3,13104,CONCHALÍ,11427.335938,11.109763,126955,61877,65078,37759,0.512607,0.487393,...,7941,5988,4801,4250,3533,2521,1625,552,127,43
4,13105,EL BOSQUE,11344.626953,14.324402,162505,79372,83133,47941,0.511572,0.488428,...,10091,8212,6348,5352,3737,2362,1473,452,104,42


In [924]:
"""
Get demographic information for communes in Santiago.
"""
df_C = df_demos.copy()
df_C.loc[:,"COMUNA"] = df_C.loc[:,"COMUNA"].apply(get_com_sid)
df_C = df_C.dropna().sort_values("COMUNA")
df_C.drop(["COMUNA", "NOM_COMUNA", "NOM_COMUNA.1"], axis=1, inplace=True)

# Get demographics feature matrix
C = df_C.to_numpy()

In [925]:
# Demographics features over 51 communes in Santiago
print(C.shape)

(51, 31)


### Feature Extraction

In [926]:
# Feature extraction
def extract_features(A, y, C):
    """
    Extracts feature vectors for each sequential input.
    Returns a matrix of all features X[N,L,T] where X[:,:,t] is matrix of features for data at time t.
    """ 
    n = 7                    # num of added features
    L = C.shape[1] + n       # feature dimension
    N = C.shape[0]           # num of communes
    
    # get overlap of mobility data and infections data - hardcoded
    A = A[:,:,5:]
    y = y[:,:4]
    
#     L = 7
    
    # extract features
    T = y.shape[1]
    X = np.zeros((N, L, T))
    for t in range(T):
        At = A[:,:,t]
        yt = y[:,t]
        
        x1 = np.ones(N)
        x2 = np.diag(At)
        x3 = np.dot(At.T, np.ones(N))
        x4 = np.dot(At, np.ones(N))
        x5 = yt
        x6 = np.dot(At.T, yt)
        x7 = np.dot(At, yt)
          
        Xt = np.stack((x1,x2,x3,x4,x5,x6,x7), axis=-1)
        Xt = np.column_stack((Xt, C))
        X[:,:,t] = Xt
    
    return X

def collect_data(X, y):
    """
    Collects data into a big 'ol matrix and output matrix to feed into a regression method.
    """
    N = X.shape[0]           # num of communes
    L = X.shape[1]           # feature dimension
    T = X.shape[2]           # number of time steps
    K = 2                    # lag
                  
    Z = np.zeros((N*(T-K), L*K))
    for t in range(T-K):
        Z[N*t:N*(t+1),:] = X[:,:,t:t+K].transpose([0,2,1]).reshape((N, L*K))
                 
    Y = np.zeros(N*(T-K))
    for t in range(T-K):
        Y[N*t:N*(t+1)] = y[:,t+1].reshape((N))
        
    return Z, Y

In [927]:
def timeseries_train_test_split(X, Y):
    """
    Splits data into a train and test data.
    """
    N = 51           # num of communes
    T = 4            # number of time steps
    K = 2            # lag
    n = 1            # num test points
    
    X_train = X[:N*(T-K-n),:]
    X_test = X[N*(T-K-n):,:]
    y_train = Y[:N*(T-K-n)]
    y_test = Y[N*(T-K-n):]
    
    return X_train, y_train, X_test, y_test

In [928]:
### Plot forecast
def train_test_plot(model, X_train, X_test):
    """
    This will plot the actual values of data against the one fitted by the model.
    """
    fig, ax = plt.subplots(figsize=(12,4))
    colors = sns.color_palette("deep", 8)
    
    yvalues = pd.DataFrame(y_test)
    
    forecasted = list(model.predict(X_test)) # Use the model fit on features data
    
    df_fcast = pd.DataFrame({"date": list(yvalues.index), "cpi_fcast": forecasted})
    df_fcast = df_fcast.set_index("date")
    
    df = pd.merge(yvalues, df_fcast, left_index=True, right_index=True)

    df["cpi_fcast"].plot(ax=ax, legend=True, linewidth=2.5, linestyle="dashed", color="forestgreen") # fitted
    df["cpi_diff"].plot(ax=ax, legend=True, linewidth=1.5, linestyle="solid", color="salmon") # Actual values
    
    ax.set_title("Actual vs. Model")
    ax.set_ylabel("Number of Infections")
    ax.legend(["Fitted","Actual"])

### Linear Regression

In [929]:
"""
Extract features
"""
Z = extract_features(A, y, C)
X1, y1 = collect_data(Z, y)
X_train, y_train, X_test, y_test = timeseries_train_test_split(X1, y1)

print(Z.shape, X1.shape, y1.shape)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(51, 38, 4) (102, 76) (102,)
(51, 76) (51,) (51, 76) (51,)


In [930]:
"""
Run linear regression
"""
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)
np.sqrt(metrics.mean_squared_error(y_test, reg.predict(X_test)))

8.509104067157643e-09

In [931]:
# exp_y_test = np.exp(y_test) - 1
# exp_y_pred = np.exp(reg.predict(X_test)) - 1
# print(np.linalg.norm(exp_y_test - exp_y_pred))
# print(exp_y_test)
# print(exp_y_pred)
# print(exp_y_pred.mean())

In [932]:
print(reg.predict(X_test).mean())
print(np.linalg.norm(y_test-reg.predict(X_test)))

253.82352941240052
6.076715768662918e-08


### LASSO

In [933]:
"""
Run LASSO
"""
lasso = linear_model.LassoCV(cv=model_selection.TimeSeriesSplit(),alphas=None, tol = 10000, normalize=True) 
cv_lasso = lasso.fit(X_train, y_train)
optimal_alpha = cv_lasso.alpha_

lasso2 = linear_model.Lasso(alpha=optimal_alpha, normalize=True)
lasso2.fit(X_train, y_train)
np.sqrt(metrics.mean_squared_error(y_test, lasso2.predict(X_test)))

/Users/gael/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:784: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)


0.3715937583534762

In [935]:
# Feature selection

fnames = []
for i in range(Z.shape[1]):
    if i < 7:
        fnames.append('x'+str(i+1)+'_2')
    else:
        fnames.append('c'+str(i-6)+'_2')
for i in range(Z.shape[1]):
    if i < 7:
        fnames.append('x'+str(i+1)+'_1')
    else:
        fnames.append('c'+str(i-6)+'_1')

# print(lasso2.coef_)
for pair in zip(fnames, lasso2.coef_):
    print(pair)
# lasso_coefs = pd.DataFrame({"features":list(X_train), "coef": lasso2.coef_})
# lasso_coefs = lasso_coefs[lasso_coefs.coef != 0.0]
# lasso_coefs.sort_values("coef", ascending=False)

('x1_2', 0.0)
('x2_2', 0.0)
('x3_2', 0.0)
('x4_2', 0.0)
('x5_2', 0.0008536259649941892)
('x6_2', 2.4405985339055815e-09)
('x7_2', 2.495975131010658e-10)
('c1_2', 0.0)
('c2_2', -0.0)
('c3_2', 0.0)
('c4_2', 0.0)
('c5_2', 0.0)
('c6_2', 0.0)
('c7_2', 0.0)
('c8_2', -0.0)
('c9_2', -0.0)
('c10_2', 0.0)
('c11_2', 0.0)
('c12_2', 0.0)
('c13_2', 0.0)
('c14_2', 0.0)
('c15_2', 0.0)
('c16_2', 0.0)
('c17_2', 0.0)
('c18_2', 0.0)
('c19_2', 0.0)
('c20_2', 0.0)
('c21_2', 0.0)
('c22_2', 0.0)
('c23_2', 0.0)
('c24_2', 0.0)
('c25_2', 0.0)
('c26_2', 0.0)
('c27_2', 0.0)
('c28_2', 0.0)
('c29_2', 0.0)
('c30_2', 0.0)
('c31_2', 0.0)
('x1_1', 0.0)
('x2_1', 0.0)
('x3_1', 0.0)
('x4_1', 0.0)
('x5_1', 0.9979978435283269)
('x6_1', 0.0)
('x7_1', 0.0)
('c1_1', 0.0)
('c2_1', -0.0)
('c3_1', 0.0)
('c4_1', 0.0)
('c5_1', 0.0)
('c6_1', 0.0)
('c7_1', 0.0)
('c8_1', -0.0)
('c9_1', -0.0)
('c10_1', 0.0)
('c11_1', 0.0)
('c12_1', 0.0)
('c13_1', 0.0)
('c14_1', 0.0)
('c15_1', 0.0)
('c16_1', 0.0)
('c17_1', 0.0)
('c18_1', 0.0)
('c19_1', 0

###### The following implementations are not finished yet...

### XGBoost

In [ ]:
X_train, y_train, X_test, y_test = timeseries_train_test_split(X=X, y=y)


scaler = preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

xgb = XGBRegressor()
xgb.fit(X_train_scaled, y_train)

X_test_scaled = scaler.transform(X_test)

train_test_plot(model=xgb, X_train=X_train_scaled, X_test=X_test_scaled)


### Neural Network

In [ ]:
X_train, y_train, X_test, y_test = timeseries_train_test_split(X=X, y=y)

reg = neural_network.MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
reg.fit(X, y)

train_test_plot(model=reg, X_train=X_train_scaled, X_test=X_test_scaled)